In [65]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.utils import all_estimators

df = pd.read_csv(r'data/data_processed.csv')
df_cat = pd.read_csv(r'data/data_categorical.csv')

df_good = pd.concat([df_cat, df["MonthlyIncome"], df["BusinessTravel"], df["StockOptionLevel"], df["DistanceFromHome"]], axis=1)

# print(df_good)
# copy attrition and drop from dataframe
JAtt = df["Attrition"].values

df_good = df_good.drop(["Unnamed: 0"], axis=1)
df_good = df_good.drop(['Attrition'], axis=1)
# print(df_good)

# splitting new df
df_train = df_good.iloc[:1200,:]
df_val = df_good.iloc[1200:,:]
# splitting Jatt into new sizes
JAtt_train = JAtt[:1200]
JAtt_val = JAtt[1200:]

from sklearn.ensemble import AdaBoostClassifier


maxiter = np.ceil(10**6 / 1200)

# clf = AdaBoostClassifier(n_estimators=300, random_state=0, learning_rate=1)
clf = make_pipeline(StandardScaler(), SGDClassifier(loss="hinge", penalty="l2", max_iter=maxiter, tol=1e-5))

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
clf = GridSearchCV(clf, parameters)
clf.fit(df_train, JAtt_train)

y_pred = clf.predict(df_val)

from sklearn import metrics

print(metrics.classification_report(JAtt_val, y_pred))
print(metrics.accuracy_score(JAtt_val, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.97      0.91       221
           1       0.65      0.22      0.33        49

    accuracy                           0.84       270
   macro avg       0.75      0.60      0.62       270
weighted avg       0.81      0.84      0.80       270

0.837037037037037
